In [1]:
import numpy as np
import random
from skimage import draw, io
from skimage.draw import polygon
from math import pi, acos

from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.utils.fixes import signature

ModuleNotFoundError: No module named 'skimage'

## Target

In [ ]:
dim = 50  # the width and height of the image
half_dim = int(dim / 2)
no_targets = 10  # the number of targets
no_ligands = 100  # the number of ligands
no_shapes = 4

In [ ]:
# Target & Ligand Images: image + identifier(1:target; 0:ligand)
images = np.zeros((no_ligands*no_targets,dim,dim*2,1),dtype='int8')

In [ ]:
results = np.zeros((no_ligands*no_targets,1),dtype='int8')

In [ ]:
# Generation of Rectangular Images
for i in range(no_targets):
    # generate each target image
    target_rec = np.concatenate((np.zeros((dim, half_dim)), np.ones((dim, half_dim))), axis=1)
    x_target = random.randint(0, half_dim)
    y1_target = random.randint(0, half_dim)
    y2_target = random.randint(y1_target + 5, dim)
    target_rec[y1_target:y2_target, x_target:half_dim] = 1
    
    images[i*no_ligands:(i+1)*no_ligands,:,:dim,0] = target_rec
    
    # generate rectangular ligands 
    for j in range(0, no_ligands//no_shapes):
        ligand_rec = np.ones((dim, dim))
        x1_ligand = random.randint(0, half_dim)
        x2_ligand = random.randint(x1_ligand + 5, dim)
        y1_ligand = random.randint(0, half_dim)
        y2_ligand = random.randint(y1_ligand + 5, dim)
        ligand_rec[y1_ligand:y2_ligand, x1_ligand:(x1_ligand+x2_ligand)//2] = 0

        images[i*no_ligands+j, :,dim:,0] = ligand_rec
        
        # put labels on the rectangular ligands
        results[i*no_ligands+j] = 1

    # generate triangular ligands 
    for k in range(no_ligands//no_shapes, no_ligands):
        ligand_tri = np.ones((dim,dim))
        r1,r2,r3 = [random.randrange(int(dim*(1/2)), dim), random.randrange(0, int(dim*(1/2))), random.randrange(int(dim*(1/2)), dim)]
        c1,c2,c3 = [random.randrange(0, int(dim*(1/3))), random.randrange(int(dim*(1/3)),int(dim*(2/3))), random.randrange(int(dim*(2/3)),dim) ]
        r = r1,r2,r3,r1
        c = c1,c2,c3,c1
        rr, cc = polygon(r, c)
        ligand_tri[rr, cc] = 0
        
        images[i*no_ligands+k, :,dim:,0] = ligand_tri

        # put labels on the rectangular ligands
        results[i*no_ligands+k] = 0
        
    # generate circle ligands 

    # generate star ligands 
        
        

# Preparation for CNN

In [ ]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

### Combine two data sets

In [ ]:
images = images
results = results
# images = np.concatenate((rectangular_images,Triangular_images),axis=0)
# results = np.concatenate((rectangular_results,Triangular_results),axis=0)

# target_criteria = Triangular_target_angles
# ligand_criteria = Triangular_ligand_angles
# target_criteria = rectangular_target_heights + Triangular_target_angles
# ligand_criteria = rectangular_ligand_heights + Triangular_ligand_angles
print("result shape = " + str(results.shape))

class_names = ['inactive','active']

In [ ]:
# Train set
index_train_data = random.sample(range(0,no_ligands*no_targets),int(no_ligands*no_targets*0.8))
# index_train_data = random.sample(range(0,2*no_ligands*no_targets),int(2*no_ligands*no_targets*0.8))
train_images = np.asarray([images[i] for i in index_train_data])
train_labels = np.asarray([results[i] for i in index_train_data])
# train_target_criteria = np.asarray([target_criteria[i] for i in index_train_data])
# train_ligand_criteria = np.asarray([ligand_criteria[i] for i in index_train_data])

# Test set
index_test_data = list(set(range(no_ligands*no_targets)).difference(set(index_train_data)))
# index_test_data = list(set(range(2*no_ligands*no_targets)).difference(set(index_train_data)))
test_images = np.asarray([images[i] for i in index_test_data])
test_labels = np.asarray([results[i] for i in index_test_data])
# test_target_criteria = np.asarray([target_criteria[i] for i in index_test_data])
# test_ligand_criteria = np.asarray([ligand_criteria[i] for i in index_test_data])

print("train_images shape: " + str(train_images.shape))
print("train_labels shape: " + str(train_labels.shape))
print("test_images shape: " + str(test_images.shape))
print("test_labels shape: " + str(test_labels.shape))

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(kernel_size=(5,5),filters = 128, input_shape=(50, 100,1),activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    keras.layers.Conv2D(kernel_size=(3,3),filters = 256, activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)),
    keras.layers.Conv2D(kernel_size=(3,3),filters = 256, activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2), strides=(1,1)),
    keras.layers.Conv2D(kernel_size=(3,3),filters = 256, activation='relu'),
    keras.layers.MaxPooling2D(pool_size = (2,2), strides=(1,1)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(2,activation='softmax',activity_regularizer=keras.regularizers.l2())
])
model.summary()

### Fit the model to the train data

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

In [ ]:
prediction_scores_test = model.predict(test_images)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, _ = roc_curve(1-test_labels, prediction_scores_test[:, 0])
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Compute PR curve and AP for each class
precision = dict()
recall = dict()
average_precision = dict()
precision, recall, _ = precision_recall_curve(1-test_labels, prediction_scores_test[:, 0])
average_precision = average_precision_score(1-test_labels, prediction_scores_test[:, 0])

In [ ]:
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
#plt.figure(figsize=(10,10))
lw = 2
plt.plot(recall, precision,'-',color='darkorange',label='PR curve (avg prec = %0.2f)' % average_precision)
#plt.step(recall[2], precision[2],color='darkorange',where='post',
#         label='PR curve (avg prec = %0.2f)' % average_precision[2])
plt.fill_between(recall, precision, alpha=0.2, color='darkorange')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve example')
plt.legend(loc="lower right")
plt.show()

### incorrect predictions

In [ ]:
Predictions_train = model.predict_classes(train_images)

In [ ]:
indices_incorrect_predictions_in_train = [i for i, x in enumerate(list(Predictions_train == train_labels.reshape(len(train_labels)))) if x == False ]
print("Total Number of Incorrect Predictions among Train data : ", len(indices_incorrect_predictions_in_train))

In [ ]:
for k, idx in enumerate(indices_incorrect_predictions_in_train):
    plt.imshow(train_images[idx,:,:,0])
    plt.show()
    print('Real Label : ', train_labels[idx])
    print('Predicted Label : ', Predictions_train[idx])
#     print('Target Criteria : ', train_target_criteria[idx])
#     print('Ligand Criteria : ', train_ligand_criteria[idx])
    if k > 30:
        break